In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
from scipy.stats import pearsonr

from qp_sim import *
from qp_fit import *
from utils import *

import warnings
import multiprocessing as mp
import time
warnings.filterwarnings('ignore')

In [ ]:
n_trials = 1200 
n_agent = 2000

t_parameters = np.zeros(shape=(n_agent,3))
all_data = [] 

for agent in range(n_agent):
    
    reward_probs = create_reward_probs(2,n_trials)
    param = configuration_parameters()    
    sim = qp_sim(
                agent,
                param,
                n_trials,
                reward_probs,
                probability_to_switch_parameters=0.005,
                max_change=np.random.choice([2,3,4])
        )
    
    df = pd.DataFrame(sim)
    all_data.append(df)
    t_parameters[agent] = list(param.values())
    
    
df_all_agent = pd.concat(all_data)    
df_all_agent = df_all_agent.reset_index()
df_all_agent = df_all_agent.drop(columns=['index'])  

In [ ]:
# create a list of our conditions
x = 0.2/3
conditions = [
                (df_all_agent['alpha'] <= x), # 0
                (df_all_agent['alpha'] > 1*x) & (df_all_agent['alpha'] <= 2*x), # 1
                (df_all_agent['alpha'] > 2*x) & (df_all_agent['alpha'] <= 3*x) # 2
    ]

# create a list of the values we want to assign for each condition
values = [0,1,2] 

# create a new column and use np.select to assign values to it using our lists as arguments
df_all_agent['alpha_categorical'] = np.select(conditions, values)



# create a list of our conditions
x = 10/5
conditions = [
                (df_all_agent['beta'] <= x), # 0
                (df_all_agent['beta'] > 1*x) & (df_all_agent['beta'] <= 2*x), # 1
                (df_all_agent['beta'] > 2*x) & (df_all_agent['beta'] <= 3*x), # 2
                (df_all_agent['beta'] > 3*x) & (df_all_agent['beta'] <= 4*x), # 3
                (df_all_agent['beta'] > 4*x) # 4
            ]

# create a list of the values we want to assign for each condition
values = [0,1,2,3,4]

# create a new column and use np.select to assign values to it using our lists as arguments
df_all_agent['beta_categorical'] = np.select(conditions, values)

# create a list of our conditions
conditions = [
                (df_all_agent['kappa'] < -.3) , # 0
                (df_all_agent['kappa'] > -.3) & (df_all_agent['kappa'] <= -.1), # 1
                (df_all_agent['kappa'] > -.1) & (df_all_agent['kappa'] <=  .1), # 2
                (df_all_agent['kappa'] >  .1) & (df_all_agent['kappa'] <=  .3), # 3
                (df_all_agent['kappa'] >  .3)  # 4
            ] 

# create a list of the values we want to assign for each condition
values = [0,1,2,3,4]  

# create a new column and use np.select to assign values to it using our lists as arguments
df_all_agent['kappa_categorical'] = np.select(conditions, values)

In [ ]:
df_all_agent[['agent','block','trial',
              'action','reward','alpha','beta',
              'kappa','alpha_categorical','beta_categorical',
              'kappa_categorical']].to_csv('../data/artificial_trainset_2000.csv',index=False)